In [1]:
import pandas as pd

In [2]:
df = pd.read_excel(r'data/etf - 30 week.xlsx')

In [3]:
# Converting data from df to weekly format

'''
Abbreviation for columns
N - NiftyBees
G - GoldBees
L - LiquidBees
B - Begin
E - End
'''

weekly_col = ['WB','WE','NB','NE','GB','GE','LB','LE']
weekly = pd.DataFrame(columns=weekly_col)
curr_week = -1
df_prev_week = 0
# cw_data - current week data
cw_data = dict.fromkeys(weekly_col,0.0)

for row in range(len(df)):
    # cw_ds - current week dataset
    cw_ds = df.iloc[row]
    if df_prev_week == cw_ds[1]:
        cw_data['WE'] = cw_ds[0]
        cw_data['NE'] = cw_ds[3]
        cw_data['GE'] = cw_ds[4]
        cw_data['LE'] = cw_ds[5]
    else:
        weekly.loc[curr_week] = cw_data
        curr_week += 1
        cw_data['WB'] = cw_data['WE'] = cw_ds[0]
        cw_data['NB'] = cw_data['NE'] = cw_ds[3]
        cw_data['GB'] = cw_data['GE'] = cw_ds[4]
        cw_data['LB'] = cw_data['LE'] = cw_ds[5]
    df_prev_week = cw_ds[1]

weekly.drop([-1], inplace=True)
print(weekly)

                      WB                   WE       NB       NE       GB  \
0    2012-12-03 00:00:00  2012-12-07 00:00:00   590.84   598.91  2991.00   
1    2012-12-10 00:00:00  2012-12-14 00:00:00   594.67   592.20  2961.80   
2    2012-12-17 00:00:00  2012-12-21 00:00:00   590.02   588.87  2947.00   
3    2012-12-24 00:00:00  2012-12-28 00:00:00   589.23   594.78  2917.90   
4    2012-12-31 00:00:00  2013-01-04 00:00:00   594.57   605.65  2903.10   
5    2013-01-07 00:00:00  2013-01-11 00:00:00   603.21   599.27  2904.90   
6    2013-01-14 00:00:00  2013-01-18 00:00:00   607.22   610.52  2889.80   
7    2013-01-21 00:00:00  2013-01-25 00:00:00   614.37   611.44  2889.90   
8    2013-01-28 00:00:00  2013-02-01 00:00:00   612.69   606.07  2891.65   
9    2013-02-04 00:00:00  2013-02-08 00:00:00   602.10   592.77  2867.45   
10   2013-02-11 00:00:00  2013-02-15 00:00:00   594.64   592.68  2883.75   
11   2013-02-18 00:00:00  2013-02-22 00:00:00   593.80   593.15  2831.60   
12   2013-02

In [4]:
# simulating trade and building trade book

initial_funds = 10000.00
fund_bal = initial_funds
trade_book = pd.DataFrame(columns = ['Date','Scrip','Value','Quantity','Amount'])
# trade book entry
tb_entry = dict.fromkeys(['Date','Scrip','Value','Quantity','Amount'])
period = 30
nifty_roc = gold_roc = 0
entry_num = 0

def roc_calc(col):
    initial = weekly.loc[week_num - period][col]
    final = weekly.loc[week_num - 1][col]
    return (final - initial) * 100 / initial

def invest_in(scrip):
    
    def entry_scrip():
        global entry_num, fund_bal
        tb_entry['Date'] = weekly.WB.values[week_num]
        tb_entry['Scrip'] = scrip
        tb_entry['Value'] = weekly[scrip+'B'].values[week_num]
        tb_entry['Quantity'] = int(fund_bal / tb_entry['Value'])
        tb_entry['Amount'] = tb_entry['Value'] * tb_entry['Quantity'] * -1
        fund_bal += tb_entry['Amount']
        print(tb_entry, fund_bal)
        trade_book.loc[entry_num] = tb_entry
        entry_num += 1
    def exit_scrip():
        global entry_num, fund_bal
        tb_entry['Date'] = weekly.WB.values[week_num]
        tb_entry['Scrip'] = trade_book.Scrip.values[-1]
        tb_entry['Value'] = weekly[trade_book.Scrip.values[-1]+'B'].values[week_num]
        tb_entry['Quantity'] = trade_book.Quantity.values[-1]
        tb_entry['Amount'] = tb_entry['Value'] * tb_entry['Quantity']
        fund_bal += tb_entry['Amount']
        print(tb_entry, fund_bal)
        trade_book.loc[entry_num] = tb_entry
        entry_num += 1
    
    if entry_num == 0:
        entry_scrip()
    elif scrip != trade_book.Scrip.values[-1]:
        exit_scrip()
        entry_scrip()
        

for week_num in range(period,len(weekly)):
    nifty_roc = roc_calc('NE')
    gold_roc = roc_calc('GE')
    print("{:8.2f} {:8.2f}".format(nifty_roc, gold_roc))
    if nifty_roc > 3 or  gold_roc > 3:
        invest_in('N' if nifty_roc > gold_roc else 'G')
    else:
        invest_in('L')
        
print(trade_book)

   -1.54   -19.01
{'Date': Timestamp('2013-07-01 00:00:00'), 'Scrip': 'L', 'Value': 999.99000000000001, 'Quantity': 10, 'Amount': -9999.8999999999996} 0.1
   -0.26   -15.67
    2.64   -13.52
    2.13   -12.87
   -2.19   -10.71
   -3.48   -10.36
   -7.61    -9.26
   -8.45    -1.21
   -8.94     1.09
   -7.08     1.01
   -3.27     0.14
   -0.33    -0.31
    5.55     1.09
{'Date': Timestamp('2013-09-23 00:00:00'), 'Scrip': 'L', 'Value': 1000.0, 'Quantity': 10, 'Amount': 10000.0} 10000.1
{'Date': Timestamp('2013-09-23 00:00:00'), 'Scrip': 'N', 'Value': 596.76999999999998, 'Quantity': 16, 'Amount': -9548.3199999999997} 451.78
   -1.49     0.86
{'Date': Timestamp('2013-09-30 00:00:00'), 'Scrip': 'N', 'Value': 582.24000000000001, 'Quantity': 16, 'Amount': 9315.8400000000001} 9767.62
{'Date': Timestamp('2013-09-30 00:00:00'), 'Scrip': 'L', 'Value': 1000.0, 'Quantity': 9, 'Amount': -9000.0} 767.62
    2.09    -0.04
    8.82    -0.45
{'Date': Timestamp('2013-10-14 00:00:00'), 'Scrip': 'L', 'Value

In [26]:
ltp = weekly[trade_book.Scrip.values[-1]+'E'].values[-1]
print(ltp)
print(trade_book.Quantity.values[-1])
print(fund_bal)
print(ltp * trade_book.Quantity.values[-1])
print(ltp * trade_book.Quantity.values[-1] + fund_bal)

1069.46
14
244.99
14972.44
15217.43
